# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [22]:
grp = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
grp

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [66]:
grp2 = pd.pivot_table(grp,columns=['CustomerID'],values=['Quantity'],index=['ProductName'],fill_value=0)
grp2

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   
...                                    ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                     0     0     0     0     0     0   
Yoghurt Tubes                            0     0     0     0     0     0   
Yogurt - Blueberry, 175 Gr               0     1     0     0     0     0   
Yogurt - French Vanilla                  1     0     0     1     0     0   
Zucchini - Yellow                        0     0     0     0     0     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   
...                                 ...   ...   ...   ...  ...   ...   ...   
Yeast Dry - Fermipan                  0     0     0     0  ...     0     0   
Yoghurt Tubes                         0     0     2     0  ...     0     0   
Yogurt - Blueberry, 175 Gr            0     0     0     0  ...    25     0   
Yogurt - French Vanilla               2     0     0     1  ...     0     0   
Zucchini - Yellow                     0     0     0     0  ...     0     0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   
...                                 ...   ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                  0    25     0     0     0     0     0   
Yoghurt Tubes                         0     0     0     0     0    25     0   
Yogurt - Blueberry, 175 Gr            0    25     0     0     0     0     0   
Yogurt - French Vanilla              25     0     0     0     0     0     0   
Zucchini - Yellow                     0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  
...                                 ...  
Yeast Dry - Fermipan                  0  
Yoghurt Tubes                         0  
Yogurt - Blueberry, 175 Gr            0  
Yogurt - French Vanilla              25  
Zucchini - Yellow                     0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
from scipy.spatial.distance import pdist, squareform

distances = pd.DataFrame(1/(1 + squareform(pdist(grp.T, 'euclidean'))), 
                         index=grp.columns, columns=grp.columns)
distances

Quantity                                          \
CustomerID              33        200       264       356       412     
         CustomerID                                                     
Quantity 33          1.000000  0.077421  0.087047  0.081800  0.080634   
         200         0.077421  1.000000  0.078448  0.076435  0.073693   
         264         0.087047  0.078448  1.000000  0.080070  0.081800   
         356         0.081800  0.076435  0.080070  1.000000  0.076435   
         412         0.080634  0.073693  0.081800  0.076435  1.000000   
...                       ...       ...       ...       ...       ...   
         97928       0.004515  0.004521  0.004543  0.004526  0.004530   
         98069       0.004583  0.004614  0.004595  0.004578  0.004578   
         98159       0.004355  0.004367  0.004365  0.004365  0.004367   
         98185       0.004167  0.004166  0.004179  0.004175  0.004177   
         98200       0.004333  0.004335  0.004333  0.004339  0.004349   

                                                                       ...  \
CustomerID              464       477       639       649       669    ...   
         CustomerID                                                    ...   
Quantity 33          0.082709  0.074573  0.083020  0.081503  0.080070  ...   
         200         0.075255  0.075956  0.076435  0.077674  0.076923  ...   
         264         0.080350  0.076923  0.080634  0.082100  0.078448  ...   
         356         0.078187  0.075025  0.082403  0.077171  0.075956  ...   
         412         0.078711  0.075025  0.082403  0.078187  0.078448  ...   
...                       ...       ...       ...       ...       ...  ...   
         97928       0.004533  0.004534  0.004526  0.004528  0.004523  ...   
         98069       0.004578  0.004589  0.004581  0.004585  0.004592  ...   
         98159       0.004363  0.004366  0.004363  0.004369  0.004365  ...   
         98185       0.004167  0.004168  0.004176  0.004172  0.004161  ...   
         98200       0.004347  0.004338  0.004335  0.004329  0.004347  ...   

                                                                       \
CustomerID              97697     97753     97769     97793     97900   
         CustomerID                                                     
Quantity 33          0.004811  0.004669  0.004412  0.005019  0.004312   
         200         0.004824  0.004681  0.004431  0.005047  0.004311   
         264         0.004822  0.004674  0.004416  0.005035  0.004322   
         356         0.004816  0.004671  0.004416  0.005038  0.004310   
         412         0.004810  0.004702  0.004414  0.005034  0.004318   
...                       ...       ...       ...       ...       ...   
         97928       0.003509  0.003509  0.003300  0.003653  0.003579   
         98069       0.003509  0.003509  0.003369  0.003509  0.003496   
         98159       0.003509  0.003565  0.003322  0.003431  0.003322   
         98185       0.003311  0.003162  0.003255  0.003406  0.003172   
         98200       0.003418  0.003345  0.003381  0.003444  0.003288   

                                                                       
CustomerID              97928     98069     98159     98185     98200  
         CustomerID                                                    
Quantity 33          0.004515  0.004583  0.004355  0.004167  0.004333  
         200         0.004521  0.004614  0.004367  0.004166  0.004335  
         264         0.004543  0.004595  0.004365  0.004179  0.004333  
         356         0.004526  0.004578  0.004365  0.004175  0.004339  
         412         0.004530  0.004578  0.004367  0.004177  0.004349  
...                       ...       ...       ...       ...       ...  
         97928       1.000000  0.003638  0.003418  0.003255  0.003245  
         98069       0.003638  1.000000  0.003469  0.003345  0.003381  
         98159       0.003418  0.003469  1.000000  0.003277  0.003266  
         98185    

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [184]:
similarities = pd.DataFrame(distances[('Quantity',33)].sort_values(ascending=False)[1:6])
display(similarities)
indexes = [y for x,y in similarities.index]
print(indexes)

Quantity
                           33
         CustomerID          
Quantity 264         0.087047
         3535        0.087047
         3317        0.087047
         2503        0.085983
         3305        0.085638

[264, 3535, 3317, 2503, 3305]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [186]:
five_best = grp.loc[indexes]
five_best

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [187]:
ranked_products = five_best.sort_values('Quantity',ascending=False)
ranked_products

Quantity
CustomerID ProductName                                
2503       Butter - Unsalted                         2
           Bar - Granola Trail Mix Fruit Nut         2
3305       Wine - Ej Gallo Sierra Valley             2
2503       Knife Plastic - White                     2
264        Guinea Fowl                               2
...                                                ...
2503       Soup - Campbells, Cream Of                1
           Soup - Canadian Pea, Dry Mix              1
           Tea - Decaf Lipton                        1
           Tia Maria                                 1
3535       Yoghurt Tubes                             1

[270 rows x 1 columns]

In [146]:
ranked_products.merge(grp2[('Quantity',33)],on='ProductName').head(5)

,Quantity,"(Quantity, 33)"
ProductName,,
Butter - Unsalted,2,0
Butter - Unsalted,1,0
Bar - Granola Trail Mix Fruit Nut,2,0
Wine - Ej Gallo Sierra Valley,2,0
Wine - Ej Gallo Sierra Valley,1,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [214]:
recommendations = {}
customers_id = list(set(data['CustomerID']))
for ID in customers_id:
        similarities = distances[('Quantity',ID)].sort_values(ascending=False)[1:6]
        indexes = set([y for x,y in similarities.index])
        data2 = grp2.loc[indexes]
        data2 = data2.sort_values('Quantity',ascending=False) 
        data2 = data2.merge(grp2[('Quantity',33)],on='ProductName').groupby(by='ProductName').sum()               
        data2 = list(data2.ProductName)
        recommendations[ID] = data2
        break

KeyError: "None of [Int64Index([28867, 22982, 23176, 20648, 23001], dtype='int64', name='ProductName')] are in the [index]"

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
finaldf = pd.DataFrame(recommendations)

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
distances = pd.DataFrame(1/(1 + squareform(pdist(grp.T, 'correlation'))), 
                         index=grp.columns, columns=grp.columns)
recommendations = {}
customers_id = list(set(data['CustomerID']))
for ID in customers_id:
        similarities = distances[('Quantity',ID)].sort_values(ascending=False)
        indexes = [y for x,y in similarities.index]
        data2 = grp2.loc[indexes]
        data2 = data2.sort_values('Quantity',ascending=False)
        data2 = data2.merge(grp2[('Quantity',33)],on='ProductName')
        data2 = list(data2.ProductName)
        recommendations[ID] = data2
        break
recommendations